# DATASETS

In [28]:
import bootcampviztools as bt
import hashlib
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import modulo_toolbox as mtb
import requests
import seaborn as sns
import time
import toolbox_ML as tb
import warnings

warnings.filterwarnings('ignore')

##### DEBILIDADES Y FORTALEZAS 

DC

In [29]:
# Cargar data DC
dc_df = pd.read_csv('./data/DC_extended_universe.csv')

In [56]:
# Creo nuevo data, solo con nombre
data_DC_nombre = dc_df[['nombre']]
data_DC_nombre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nombre  73 non-null     object
dtypes: object(1)
memory usage: 716.0+ bytes


In [34]:
# Guardar data en .csv
data_DC_nombre.to_csv('./data/DC_nombres.csv', index = False)

In [35]:
# Fortalezas y debilidades basadas en cómics y películas
strengths_weaknesses_DC = {
    "Aquaman": {
        "Fortalezas": ["Control sobre animales marinos", "Fuerza sobrehumana", "Durabilidad", "Habilidades de combate", "Comunicación telepática con la vida marina"],
        "Debilidades": ["Necesidad de estar en contacto con el agua regularmente", "Vulnerabilidad a la deshidratación"]
    },
    "Ares": {
        "Fortalezas": ["Fuerza y durabilidad sobrehumana", "Habilidades mágicas", "Control sobre el caos y la guerra", "Inmortalidad"],
        "Debilidades": ["Dependencia del caos y conflicto para mantener su poder"]
    },
    "Atom Smasher": {
        "Fortalezas": ["Cambio de tamaño", "Fuerza incrementada con el tamaño", "Resistencia"],
        "Debilidades": ["Difícil de controlar su tamaño", "Problemas con la coordinación al cambiar de tamaño"]
    },
    "Batman": {
        "Fortalezas": ["Inteligencia de nivel genio", "Maestro en artes marciales", "Detección avanzada", "Equipamiento y gadgets", "Habilidades de detective"],
        "Debilidades": ["Humano sin poderes sobrenaturales", "Vulnerabilidad física"]
    },
    "Black Adam": {
        "Fortalezas": ["Fuerza sobrehumana", "Velocidad", "Magia", "Vuelo", "Resistencia a la magia"],
        "Debilidades": ["Vulnerable a la magia poderosa", "Arrogancia"]
    },
    "Black Canary": {
        "Fortalezas": ["Grito sónico", "Habilidades de combate"],
        "Debilidades": ["Humana sin poderes sobrenaturales fuera de su grito"]
    },
    "Black Manta": {
        "Fortalezas": ["Habilidades de combate", "Tecnología avanzada"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Black Mask": {
        "Fortalezas": ["Estratega criminal", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Bloodsport": {
        "Fortalezas": ["Experto en armas", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Captain Boomerang": {
        "Fortalezas": ["Habilidad con boomerangs", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Cassandra Cain": {
        "Fortalezas": ["Habilidades de combate", "Experta en artes marciales"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Cheetah": {
        "Fortalezas": ["Fuerza y agilidad sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia de la transformación"]
    },
    "Cyborg": {
        "Fortalezas": ["Tecnología avanzada", "Fuerza sobrehumana", "Habilidades de hackeo"],
        "Debilidades": ["Dependencia de su tecnología"]
    },
    "Cyclone": {
        "Fortalezas": ["Control del viento", "Vuelo"],
        "Debilidades": ["Vulnerabilidad física"]
    },
    "Dark Flash": {
        "Fortalezas": ["Velocidad sobrehumana", "Reflejos rápidos"],
        "Debilidades": ["Dependencia de la Speed Force"]
    },
    "Darkseid": {
        "Fortalezas": ["Fuerza y durabilidad sobrehumana", "Habilidades mágicas", "Control sobre el caos y la guerra", "Inmortalidad"],
        "Debilidades": ["Dependencia del caos y conflicto para mantener su poder"]
    },
    "Deadshot": {
        "Fortalezas": ["Experto en armas", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Doctor Fate": {
        "Fortalezas": ["Habilidades mágicas", "Inmortalidad", "Vuelo"],
        "Debilidades": ["Dependencia del casco"]
    },
    "Doctor Sivana": {
        "Fortalezas": ["Intelecto de nivel genio", "Habilidades científicas"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Doomsday": {
        "Fortalezas": ["Fuerza y durabilidad sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerable a la magia"]
    },
    "El Diablo": {
        "Fortalezas": ["Control del fuego", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales fuera de su control de fuego"]
    },
    "Enchantress": {
        "Fortalezas": ["Habilidades mágicas", "Control mental"],
        "Debilidades": ["Dependencia de su magia"]
    },
    "Envy": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Dependencia de la envidia"]
    },
    "Faora": {
        "Fortalezas": ["Fuerza y durabilidad sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerable a la kryptonita"]
    },
    "Freddy Freeman": {
        "Fortalezas": ["Fuerza sobrehumana", "Velocidad", "Vuelo"],
        "Debilidades": ["Dependencia de la magia de Shazam"]
    },
    "General Zod": {
        "Fortalezas": ["Fuerza y durabilidad sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerable a la kryptonita"]
    },
    "Gluttony": {
        "Fortalezas": ["Fuerza sobrehumana"],
        "Debilidades": ["Dependencia de la gula"]
    },
    "Green Lantern": {
        "Fortalezas": ["Creación de construcciones de luz sólida", "Vuelo", "Viaje interestelar", "Resistencia"],
        "Debilidades": ["Vulnerabilidad al color amarillo (dependiendo de la versión)", "Dependencia del anillo y su carga"]
    },
    "Harley Quinn": {
        "Fortalezas": ["Habilidades de combate", "Inteligencia"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Hawkman": {
        "Fortalezas": ["Vuelo", "Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia de su armadura"]
    },
    "Hespera": {
        "Fortalezas": ["Habilidades mágicas"],
        "Debilidades": ["Dependencia de su magia"]
    },
    "Huntress": {
        "Fortalezas": ["Habilidades de combate", "Experta en armas"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Incubus": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Dependencia de la lujuria"]
    },
    "Joker": {
        "Fortalezas": ["Inteligencia", "Habilidades estratégicas"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Jor-El": {
        "Fortalezas": ["Inteligencia"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Judomaster": {
        "Fortalezas": ["Habilidades de combate", "Experta en artes marciales"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Karathen": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerabilidad física"]
    },
    "Katana": {
        "Fortalezas": ["Habilidades de combate", "Experta en armas"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Killer Croc": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerabilidad física"]
    },
    "King Nereus": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Necesidad de estar en contacto con el agua regularmente"]
    },
    "King Shark": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerabilidad física"]
    },
        "Lex Luthor": {
        "Fortalezas": ["Inteligencia", "Habilidades estratégicas"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Lust": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Dependencia de la lujuria"]
    },
    "Mera": {
        "Fortalezas": ["Control sobre el agua", "Habilidades de combate"],
        "Debilidades": ["Necesidad de estar en contacto con el agua regularmente"]
    },
    "Mister Zsasz": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Nam-Ek": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerable a la kryptonita"]
    },
    "Nuidis Vulko": {
        "Fortalezas": ["Inteligencia", "Habilidades estratégicas"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Ocean Master": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Necesidad de estar en contacto con el agua regularmente"]
    },
    "Parademon": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia de Darkseid"]
    },
    "Peacemaker": {
        "Fortalezas": ["Habilidades de combate", "Experto en armas"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Penthesileia": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Philippus": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Polka-Dot Man": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Queen Atlanna": {
        "Fortalezas": ["Control sobre el agua", "Habilidades de combate"],
        "Debilidades": ["Necesidad de estar en contacto con el agua regularmente"]
    },
    "Queen Hippolyta": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Inmortalidad"],
        "Debilidades": ["Dependencia de la inmortalidad"]
    },
    "Ratcatcher": {
        "Fortalezas": ["Control sobre ratas", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Rick Flag": {
        "Fortalezas": ["Habilidades de combate", "Liderazgo"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Sabbac": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades mágicas", "Vuelo"],
        "Debilidades": ["Dependencia de la magia"]
    },
    "Savant": {
        "Fortalezas": ["Intelecto de nivel genio", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Shazam": {
        "Fortalezas": ["Fuerza sobrehumana", "Velocidad", "Vuelo", "Magia"],
        "Debilidades": ["Dependencia de la magia de Shazam"]
    },
    "Slipknot": {
        "Fortalezas": ["Habilidad con cuerdas", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Sol Soria": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Starro": {
        "Fortalezas": ["Control mental", "Regeneración", "Fuerza sobrehumana"],
        "Debilidades": ["Vulnerabilidad a la separación física"]
    },
    "Steppenwolf": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Inmortalidad"],
        "Debilidades": ["Dependencia de Darkseid"]
    },
    "Supergirl": {
        "Fortalezas": ["Fuerza sobrehumana", "Velocidad", "Vuelo", "Visión de rayos X", "Invulnerabilidad", "Visión calorífica", "Aliento helado"],
        "Debilidades": ["Kryptonita", "Magia"]
    },
    "Superman": {
        "Fortalezas": ["Fuerza sobrehumana", "Velocidad", "Vuelo", "Visión de rayos X", "Invulnerabilidad", "Visión calorífica", "Aliento helado"],
        "Debilidades": ["Kryptonita", "Magia"]
    },
    "T.D.K.": {
        "Fortalezas": ["Desprendimiento de extremidades", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales fuera de su habilidad"]
    },
    "The Flash": {
        "Fortalezas": ["Velocidad sobrehumana", "Reflejos rápidos", "Intangibilidad", "Viaje en el tiempo", "Curación acelerada"],
        "Debilidades": ["Fuerzas de fricción extremas", "Dependencia de la Speed Force"]
    },
    "Vigilante": {
        "Fortalezas": ["Habilidades de combate", "Experto en armas"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "White Dragon": {
        "Fortalezas": ["Habilidades de combate", "Tecnología avanzada"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Wonder Woman (Armor of Asteria)": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Vuelo", "Agilidad", "Resistencia"],
        "Debilidades": ["Dependencia de la verdad"]
    },
    "Wrath": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Zeus": {
        "Fortalezas": ["Habilidades mágicas", "Inmortalidad", "Control del rayo"],
        "Debilidades": ["Dependencia de su magia"]
    }
}


In [39]:
# Asignar fortalezas y debilidades
def assign_strenghs_weaknesses_DC(row):
    hero_name = row['nombre']
    if hero_name in strengths_weaknesses_DC:
        return pd.Series([
            strengths_weaknesses_DC[hero_name]['Fortalezas'],
            strengths_weaknesses_DC[hero_name]['Debilidades']
        ])
    else:
        return pd.Series([['Desconocido'], ['Desconocido']])

In [40]:
dc_df[['Fortalezas', 'Debilidades']] = dc_df.apply(assign_strenghs_weaknesses_DC, axis = 1)

In [75]:
dc_df.info()
dc_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre_completo  61 non-null     object
 1   nombre           73 non-null     object
 2   universo         73 non-null     object
 3   url              73 non-null     object
 4   Intelligence     73 non-null     int64 
 5   Strength         73 non-null     int64 
 6   Speed            73 non-null     int64 
 7   Durability       73 non-null     int64 
 8   Power            73 non-null     int64 
 9   Combat           73 non-null     int64 
 10  Tier             73 non-null     int64 
 11  Tier2            9 non-null      object
 12  superpoderes     73 non-null     object
 13  heroe            73 non-null     object
 14  Fortalezas       73 non-null     object
 15  Debilidades      73 non-null     object
dtypes: int64(7), object(9)
memory usage: 9.3+ KB


,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,superpoderes,heroe,Fortalezas,Debilidades
0,Arthur Curry,Aquaman,DC Extended Universe,https://www.superherodb.com/aquaman/10-12533/,65,50,25,30,45,80,3,NaN,"['Animal Control', 'Durability', 'Stamina', 'S...",Aquaman,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...
1,Ares,Ares,DC Extended Universe,https://www.superherodb.com/ares/10-12597/,90,100,50,50,70,100,4,4,"['Accelerated Healing', 'Chain Manipulation', ...",Ares,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...
2,Albert Rothstein,Atom Smasher,DC Extended Universe,https://www.superherodb.com/atom-smasher/10-27...,60,30,25,25,40,20,2,NaN,"['Jaw Strength', 'Large Size', 'Size Changing'...",Atom Smasher,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con..."
3,Bruce Wayne,Batman,DC Extended Universe,https://www.superherodb.com/batman/10-12530/,80,13,11,15,20,95,1,NaN,"['Bloodlust', 'Dexterity', 'Disguise Mastery',...",Batman,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil..."
4,Teth-Adam,Black Adam,DC Extended Universe,https://www.superherodb.com/black-adam/10-21003/,65,100,75,45,75,75,3,3,"['Afterimage Creation', 'Agility', 'Cold Resis...",Black Adam,"[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]"


THE BOYS 

In [47]:
theboys_df = pd.read_csv('./data/The_boys.csv')

In [48]:
# Creo nuevo data, solo con nombre
data_TB_nombre = theboys_df[['nombre']]
data_TB_nombre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nombre  35 non-null     object
dtypes: object(1)
memory usage: 412.0+ bytes


In [50]:
# Guardar data en .csv
data_TB_nombre.to_csv('./data/TB_nombres.csv', index = False)

In [51]:
# Lista de fortalezas y debilidades basadas en la serie "The Boys"
strengths_weaknesses_TB = {
    "A-Train": {
        "Fortalezas": ["Velocidad sobrehumana"],
        "Debilidades": ["Problemas cardíacos", "Dependencia de Compuesto V"]
    },
    "Billy Butcher": {
        "Fortalezas": ["Habilidades de combate", "Liderazgo"],
        "Debilidades": ["Humano sin poderes sobrenaturales", "Impulsividad"]
    },
    "Billy Butcher (V24)": {
        "Fortalezas": ["Fuerza sobrehumana (temporal)", "Habilidades de combate"],
        "Debilidades": ["Efectos secundarios de V24", "Tiempo limitado de poderes"]
    },
    "Billy Butcher (tumor)": {
        "Fortalezas": ["Determinación", "Habilidades de combate"],
        "Debilidades": ["Enfermedad terminal", "Humano sin poderes sobrenaturales"]
    },
    "Black Noir (replacement)": {
        "Fortalezas": ["Habilidades de combate", "Sigilo"],
        "Debilidades": ["Desconocido"]
    },
    "Black Noir": {
        "Fortalezas": ["Habilidades de combate", "Resistencia sobrehumana"],
        "Debilidades": ["Vulnerabilidad a heridas graves", "Trauma psicológico"]
    },
    "Cate Dunlap": {
        "Fortalezas": ["Habilidades desconocidas"],
        "Debilidades": ["Desconocido"]
    },
    "Crimson Countess": {
        "Fortalezas": ["Manipulación de energía", "Vuelo"],
        "Debilidades": ["Dependencia emocional", "Humano sin invulnerabilidad"]
    },
    "Firecracker": {
        "Fortalezas": ["Piroquinesis"],
        "Debilidades": ["Desconocido"]
    },
    "Golden Boy": {
        "Fortalezas": ["Fuerza sobrehumana", "Piroquinesis"],
        "Debilidades": ["Arrogancia", "Impulsividad"]
    },
    "Homelander": {
        "Fortalezas": ["Fuerza sobrehumana", "Vuelo", "Visión calorífica", "Invulnerabilidad"],
        "Debilidades": ["Inestabilidad mental", "Narcisismo"]
    },
    "Hughie Campbell (V24)": {
        "Fortalezas": ["Fuerza sobrehumana (temporal)", "Teleportación (temporal)"],
        "Debilidades": ["Efectos secundarios de V24", "Tiempo limitado de poderes"]
    },
    "Lamplighter": {
        "Fortalezas": ["Manipulación del fuego"],
        "Debilidades": ["Dependencia emocional", "Trauma psicológico"]
    },
    "Little Cricket": {
        "Fortalezas": ["Cambio de tamaño"],
        "Debilidades": ["Vulnerabilidad física cuando es pequeña"]
    },
    "Marie Moreau": {
        "Fortalezas": ["Control de la sangre"],
        "Debilidades": ["Desconocido"]
    },
    "Mesmer": {
        "Fortalezas": ["Lectura de mentes"],
        "Debilidades": ["Vulnerabilidad física", "Humano sin poderes de combate"]
    },
    "Mother's Milk": {
        "Fortalezas": ["Habilidades de combate", "Estrategia"],
        "Debilidades": ["Humano sin poderes sobrenaturales", "Trauma emocional"]
    },
    "Naqib": {
        "Fortalezas": ["Explosión de energía"],
        "Debilidades": ["Desconocido"]
    },
    "Queen Maeve": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia emocional", "Humano sin invulnerabilidad total"]
    },
    "Ryan Butcher": {
        "Fortalezas": ["Fuerza sobrehumana", "Vuelo", "Visión calorífica"],
        "Debilidades": ["Inexperiencia", "Emocionalmente inestable"]
    },
    "Sam Riordan": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Shockwave": {
        "Fortalezas": ["Velocidad sobrehumana"],
        "Debilidades": ["Competencia con A-Train", "Dependencia de Compuesto V"]
    },
    "Sister Sage": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Soldier Boy": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Resistencia"],
        "Debilidades": ["Trauma psicológico", "Narcisismo"]
    },
    "Splinter": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Starlight": {
        "Fortalezas": ["Manipulación de la luz", "Vuelo"],
        "Debilidades": ["Dependencia emocional", "Humano sin invulnerabilidad total"]
    },
    "Stormfront": {
        "Fortalezas": ["Manipulación de electricidad", "Vuelo", "Fuerza sobrehumana"],
        "Debilidades": ["Ideología extremista", "Vulnerabilidad emocional"]
    },
    "Supe Terrorist": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Tek Knight": {
        "Fortalezas": ["Tecnología avanzada", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "The Deep": {
        "Fortalezas": ["Comunicación con la vida marina", "Habilidades de combate"],
        "Debilidades": ["Inestabilidad emocional", "Humano sin invulnerabilidad"]
    },
    "The Female": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Regeneración"],
        "Debilidades": ["Trauma emocional"]
    },
    "Translucent": {
        "Fortalezas": ["Invisibilidad", "Piel a prueba de balas"],
        "Debilidades": ["Vulnerabilidad a explosivos internos"]
    },
    "Victoria Neuman": {
        "Fortalezas": ["Explosión de cabezas"],
        "Debilidades": ["Secretismo", "Humano sin invulnerabilidad"]
    },
    "Webweaver": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Zoe Neuman": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    }
}


In [53]:
# Asignar fortalezas y debilidades
def assign_strenghs_weaknesses_Theboys(row):
    hero_name = row['nombre']
    if hero_name in strengths_weaknesses_TB:
        return pd.Series([
            strengths_weaknesses_TB[hero_name]['Fortalezas'],
            strengths_weaknesses_TB[hero_name]['Debilidades']
        ])
    else:
        return pd.Series([['Desconocido'], ['Desconocido']])

In [54]:
theboys_df[['Fortalezas', 'Debilidades']] = theboys_df.apply(assign_strenghs_weaknesses_Theboys, axis = 1)

In [74]:
theboys_df.info()
theboys_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre_completo  17 non-null     object
 1   nombre           35 non-null     object
 2   universo         35 non-null     object
 3   url              35 non-null     object
 4   Intelligence     35 non-null     int64 
 5   Strength         35 non-null     int64 
 6   Speed            35 non-null     int64 
 7   Durability       35 non-null     int64 
 8   Power            35 non-null     int64 
 9   Combat           35 non-null     int64 
 10  Tier             35 non-null     int64 
 11  Tier2            22 non-null     object
 12  superpoderes     35 non-null     object
 13  heroe            35 non-null     object
 14  Fortalezas       35 non-null     object
 15  Debilidades      35 non-null     object
dtypes: int64(7), object(9)
memory usage: 4.5+ KB


,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,superpoderes,heroe,Fortalezas,Debilidades
0,"Reginald ""Reggie"" C. Franklin",A-Train,The Boys (TV Series 2019),https://www.superherodb.com/a-train/10-21495/,60,25,25,15,35,30,1,1+,"['Agility', 'Enhanced Condition', 'Enhanced He...",A-Train,[Velocidad sobrehumana],"[Problemas cardíacos, Dependencia de Compuesto V]"
1,William Butcher,Billy Butcher,The Boys (TV Series 2019),https://www.superherodb.com/billy-butcher/10-2...,65,5,7,8,10,60,1,0+,"['Agility', 'Dexterity', 'Marksmanship', 'Mast...",Billy Butcher,"[Habilidades de combate, Liderazgo]","[Humano sin poderes sobrenaturales, Impulsividad]"
2,William Butcher,Billy Butcher (V24),The Boys (TV Series 2019),https://www.superherodb.com/billy-butcher-v24/...,65,25,8,20,50,60,1,NaN,"['Heat Generation', 'Stamina', 'Toxin and Dise...",Billy Butcher (V24),"[Fuerza sobrehumana (temporal), Habilidades de...","[Efectos secundarios de V24, Tiempo limitado d..."
3,William Butcher,Billy Butchter (tumor),The Boys (TV Series 2019),https://www.superherodb.com/billy-butchter-tum...,65,25,14,20,35,65,1,1,"['Instinctive Reaction', 'Rage Power', 'Super ...",Billy Butcher (Tumor),[Desconocido],[Desconocido]
4,NaN,Black Noir (replacement),The Boys (TV Series 2019),https://www.superherodb.com/black-noir-replace...,50,25,25,15,35,55,1,1,"['Insanity', 'Stamina', 'Super Speed', 'Super ...",Black Noir (Replacement),"[Habilidades de combate, Sigilo]",[Desconocido]


MCU

In [57]:
mcu_df = pd.read_csv('./data/MCU.csv')

In [59]:
# Creo nuevo data, solo con nombre
data_MCU_nombre = mcu_df[['nombre']]
data_MCU_nombre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nombre  176 non-null    object
dtypes: object(1)
memory usage: 1.5+ KB


In [60]:
# Guardar data en .csv
data_MCU_nombre.to_csv('./data/MCU_nombres.csv', index = False)

In [67]:
# Fortalezas y debilidades basadas en personajes del MCU
strengths_weaknesses_MCU = {
    "Agatha Harkness": {
        "Fortalezas": ["Magia oscura", "Manipulación mágica"],
        "Debilidades": ["Necesita absorber la magia de otros para mantener su poder"]
    },
    "Abomination": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Vulnerable a ataques mentales"]
    },
    "Ajak": {
        "Fortalezas": ["Curación", "Fuerza y durabilidad sobrehumanas"],
        "Debilidades": ["Vulnerable a traiciones de otros Eternos"]
    },
    "Ancient One (time stone)": {
        "Fortalezas": ["Manipulación del tiempo", "Magia avanzada"],
        "Debilidades": ["Dependencia de la Piedra del Tiempo"]
    },
    "Ancient One": {
        "Fortalezas": ["Magia avanzada", "Longevidad"],
        "Debilidades": ["Vulnerabilidad física sin la Piedra del Tiempo"]
    },
    "Ant-Man": {
        "Fortalezas": ["Cambio de tamaño", "Fuerza incrementada al reducirse"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Arishem The Judge": {
        "Fortalezas": ["Poder cósmico", "Control sobre los Celestiales"],
        "Debilidades": ["Desconocido"]
    },
    "Ayo": {
        "Fortalezas": ["Habilidades de combate", "Lealtad"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Baby Groot": {
        "Fortalezas": ["Regeneración", "Habilidades de combate"],
        "Debilidades": ["Vulnerabilidad al fuego"]
    },
    "Baron Mordo": {
        "Fortalezas": ["Magia avanzada", "Habilidades de combate"],
        "Debilidades": ["Arrogancia", "Desconfianza"]
    },
    "Baron Zemo": {
        "Fortalezas": ["Inteligencia estratégica", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Black Bolt (Earth 838)": {
        "Fortalezas": ["Voz destructiva", "Fuerza sobrehumana"],
        "Debilidades": ["Vulnerabilidad al control mental"]
    },
    "Black Bolt": {
        "Fortalezas": ["Voz destructiva", "Fuerza sobrehumana"],
        "Debilidades": ["Vulnerabilidad al control mental"]
    },
    "Black Panther": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Traje avanzado"],
        "Debilidades": ["Dependencia del traje y hierba en forma de corazón"]
    },
    "Black Widow": {
        "Fortalezas": ["Habilidades de combate", "Espionaje"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Bucky Barnes": {
        "Fortalezas": ["Fuerza sobrehumana", "Brazo biónico", "Habilidades de combate"],
        "Debilidades": ["Trauma psicológico", "Programación mental"]
    },
    "Bullseye": {
        "Fortalezas": ["Precisión letal", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Captain America": {
        "Fortalezas": ["Fuerza sobrehumana", "Liderazgo", "Habilidades de combate"],
        "Debilidades": ["Humano con límites físicos"]
    },
    "Captain America (Worthy)": {
        "Fortalezas": ["Fuerza sobrehumana", "Liderazgo", "Habilidades de combate", "Puede manejar el Mjolnir"],
        "Debilidades": ["Humano con límites físicos"]
    },
    "Captain Carter (Earth 838)": {
        "Fortalezas": ["Fuerza sobrehumana", "Liderazgo", "Habilidades de combate"],
        "Debilidades": ["Humana con límites físicos"]
    },
    "Captain Marvel": {
        "Fortalezas": ["Fuerza sobrehumana", "Vuelo", "Energía fotónica"],
        "Debilidades": ["Arrogancia", "Dependencia de sus poderes"]
    },
    "Chitauri Footsoldier": {
        "Fortalezas": ["Fuerza sobrehumana", "Armas avanzadas"],
        "Debilidades": ["Inteligencia limitada"]
    },
    "Corvus Glaive": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia de su lanza"]
    },
    "Crossbones": {
        "Fortalezas": ["Fuerza mejorada", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Cull Obsidian": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Inteligencia limitada"]
    },
    "Dar-Benn": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Daredevil": {
        "Fortalezas": ["Sentidos mejorados", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Deathlok": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Tecnología avanzada"],
        "Debilidades": ["Dependencia de su tecnología"]
    },
    "Destroyer": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia", "Armamento avanzado"],
        "Debilidades": ["Controlado externamente"]
    },
    "Doctor Strange": {
        "Fortalezas": ["Magia avanzada", "Manipulación del tiempo (con la Piedra del Tiempo)"],
        "Debilidades": ["Dependencia de artefactos mágicos"]
    },
    "Dormammu": {
        "Fortalezas": ["Poder cósmico", "Inmortalidad"],
        "Debilidades": ["Vulnerabilidad a la manipulación del tiempo"]
    },
    "Drax The Destroyer": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Inteligencia limitada"]
    },
    "Ebony Maw": {
        "Fortalezas": ["Telequinesis", "Inteligencia"],
        "Debilidades": ["Vulnerabilidad física"]
    },
    "Echo": {
        "Fortalezas": ["Habilidades de combate", "Mimetismo"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Ego": {
        "Fortalezas": ["Poder cósmico", "Manipulación de materia"],
        "Debilidades": ["Dependencia de su núcleo"]
    },
    "Elektra": {
        "Fortalezas": ["Habilidades de combate", "Sigilo"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Eternity": {
        "Fortalezas": ["Poder cósmico", "Omnisciencia"],
        "Debilidades": ["Desconocido"]
    },
    "Falcon": {
        "Fortalezas": ["Habilidades de combate", "Vuelo con traje alado"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Fandral": {
        "Fortalezas": ["Habilidades de combate", "Inmortalidad"],
        "Debilidades": ["Desconocido"]
    },
    "Fat Thor": {
        "Fortalezas": ["Fuerza sobrehumana", "Control del rayo", "Resistencia"],
        "Debilidades": ["Estado emocional inestable"]
    },
    "Fenris": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Vulnerabilidad a armas asgardianas"]
    },
    "Frigga": {
        "Fortalezas": ["Magia avanzada", "Habilidades de combate"],
        "Debilidades": ["Humana con límites físicos"]
    },
    "G'iah": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Gamora": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia emocional"]
    },
    "Gargantos": {
        "Fortalezas": ["Fuerza sobrehumana", "Múltiples tentáculos"],
        "Debilidades": ["Desconocido"]
    },
    "Ghost": {
        "Fortalezas": ["Intangibilidad", "Invisibilidad"],
        "Debilidades": ["Dependencia de su traje"]
    },
    "Ghost Rider": {
        "Fortalezas": ["Piroquinesis", "Inmortalidad"],
        "Debilidades": ["Vulnerabilidad emocional"]
    },
    "Giant-Man": {
        "Fortalezas": ["Cambio de tamaño", "Fuerza incrementada"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Gilgamesh": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Desconocido"]
    },
    "Gorr The God Butcher": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia de la Necrosword"]
    },
    "Gravik": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Groot": {
        "Fortalezas": ["Regeneración", "Fuerza sobrehumana"],
        "Debilidades": ["Vulnerabilidad al fuego"]
    },
    "Hawkeye": {
        "Fortalezas": ["Precisión", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Heimdall": {
        "Fortalezas": ["Sentidos mejorados", "Fuerza sobrehumana"],
        "Debilidades": ["Desconocido"]
    },
    "Hela": {
        "Fortalezas": ["Fuerza sobrehumana", "Control sobre armas"],
        "Debilidades": ["Dependencia de Asgard"]
    },
    "High Evolutionary": {
        "Fortalezas": ["Inteligencia avanzada", "Manipulación genética"],
        "Debilidades": ["Arrogancia"]
    },
    "Hulk (Stark Gauntlet)": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Efectos secundarios del Gauntlet"]
    },
    "Hulk": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Falta de control en su forma de Hulk"]
    },
    "Ikaris": {
        "Fortalezas": ["Vuelo", "Fuerza sobrehumana", "Proyección de energía"],
        "Debilidades": ["Vulnerabilidad emocional"]
    },
    "Iron Fist": {
        "Fortalezas": ["Puño de hierro", "Habilidades de combate"],
        "Debilidades": ["Dependencia de Chi"]
    },
    "Iron Man": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Iron Man (Mk 1)": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Iron Man (Stark Gauntlet)": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Efectos secundarios del Gauntlet"]
    },
    "Iron Man (Hulkbuster)": {
        "Fortalezas": ["Fuerza incrementada", "Tecnología avanzada"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Iron Monger": {
        "Fortalezas": ["Fuerza incrementada", "Tecnología avanzada"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Iron Spider": {
        "Fortalezas": ["Fuerza sobrehumana", "Tecnología avanzada"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Ironheart": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Jemiah The Analyzer": {
        "Fortalezas": ["Poder cósmico", "Inteligencia avanzada"],
        "Debilidades": ["Desconocido"]
    },
    "Jessica Jones": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Trauma emocional"]
    },
    "Kaecilius": {
        "Fortalezas": ["Magia avanzada", "Habilidades de combate"],
        "Debilidades": ["Arrogancia"]
    },
    "Kang the Conqueror": {
        "Fortalezas": ["Manipulación del tiempo", "Inteligencia avanzada"],
        "Debilidades": ["Dependencia de la tecnología"]
    },
    "Killian": {
        "Fortalezas": ["Poder de regeneración", "Fuerza sobrehumana"],
        "Debilidades": ["Inestabilidad emocional"]
    },
    "Killmonger": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Arrogancia"]
    },
    "Kingo": {
        "Fortalezas": ["Proyección de energía", "Inmortalidad"],
        "Debilidades": ["Dependencia emocional"]
    },
    "Kingpin": {
        "Fortalezas": ["Fuerza incrementada", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Korath The Pursuer": {
        "Fortalezas": ["Fuerza sobrehumana", "Tecnología avanzada"],
        "Debilidades": ["Desconocido"]
    },
    "Korg": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Vulnerabilidad a explosivos"]
    },
    "Kro": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Desconocido"]
    },
    "Kurse": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Desconocido"]
    },
    "Kid Loki": {
        "Fortalezas": ["Magia", "Inteligencia"],
        "Debilidades": ["Desconocido"]
    },
    "Lady Loki": {
        "Fortalezas": ["Magia", "Inteligencia"],
        "Debilidades": ["Desconocido"]
    },
    "Leo Fitz": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Leviathan": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Desconocido"]
    },
    "Loki": {
        "Fortalezas": ["Magia", "Inteligencia"],
        "Debilidades": ["Arrogancia"]
    },
    "Loki (Mind Stone)": {
        "Fortalezas": ["Magia", "Control mental"],
        "Debilidades": ["Arrogancia"]
    },
    "Loki (Keeper Of The Multiverse)": {
        "Fortalezas": ["Magia", "Inteligencia"],
        "Debilidades": ["Arrogancia"]
    },
    "Loki (Variant L1130)": {
        "Fortalezas": ["Magia", "Inteligencia"],
        "Debilidades": ["Arrogancia"]
    },
    "Loki (King of Asgard)": {
        "Fortalezas": ["Magia", "Inteligencia"],
        "Debilidades": ["Arrogancia"]
    },
    "Luke Cage": {
        "Fortalezas": ["Fuerza sobrehumana", "Piel a prueba de balas"],
        "Debilidades": ["Vulnerabilidad emocional"]
    },
    "M'baku": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Makkari": {
        "Fortalezas": ["Velocidad sobrehumana", "Inmortalidad"],
        "Debilidades": ["Desconocido"]
    },
    "Malekith (Reality Stone)": {
        "Fortalezas": ["Manipulación de la realidad", "Fuerza sobrehumana"],
        "Debilidades": ["Dependencia de la Piedra de la Realidad"]
    },
    "Malekith": {
        "Fortalezas": ["Fuerza sobrehumana", "Manipulación de la oscuridad"],
        "Debilidades": ["Desconocido"]
    },
    "Mandarin": {
        "Fortalezas": ["Habilidades de combate", "Anillos mágicos"],
        "Debilidades": ["Dependencia de los anillos"]
    },
    "Mantis": {
        "Fortalezas": ["Empatía", "Manipulación emocional"],
        "Debilidades": ["Físicamente vulnerable"]
    },
    "Mighty Thor": {
        "Fortalezas": ["Fuerza sobrehumana", "Control del rayo", "Vuelo"],
        "Debilidades": ["Cáncer debilitante"]
    },
    "MODOK": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Físicamente limitado"]
    },
    "Moon Knight": {
        "Fortalezas": ["Habilidades de combate", "Resistencia incrementada"],
        "Debilidades": ["Trastorno de identidad disociativo"]
    },
    "Mr. Fantastic (Earth 838)": {
        "Fortalezas": ["Elasticidad", "Inteligencia"],
        "Debilidades": ["Vulnerabilidad física"]
    },
    "Mr. Knight": {
        "Fortalezas": ["Habilidades de combate", "Resistencia incrementada"],
        "Debilidades": ["Trastorno de identidad disociativo"]
    },
    "Ms. America": {
        "Fortalezas": ["Fuerza sobrehumana", "Vuelo", "Viaje interdimensional"],
        "Debilidades": ["Inexperiencia"]
    },
    "Ms. Marvel": {
        "Fortalezas": ["Elasticidad", "Manipulación de energía"],
        "Debilidades": ["Inexperiencia"]
    },
    "Mysterio": {
        "Fortalezas": ["Ilusiones avanzadas", "Inteligencia"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Namor": {
        "Fortalezas": ["Fuerza sobrehumana", "Vuelo", "Control del agua"],
        "Debilidades": ["Necesita agua para mantener su fuerza"]
    },
    "Nebula": {
        "Fortalezas": ["Cibernética avanzada", "Habilidades de combate"],
        "Debilidades": ["Trauma emocional"]
    },
    "Nick Fury": {
        "Fortalezas": ["Estrategia", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Nomad": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Humano con límites físicos"]
    },
    "Odin": {
        "Fortalezas": ["Magia avanzada", "Fuerza sobrehumana", "Longevidad"],
        "Debilidades": ["Depende del Odinsleep para rejuvenecerse"]
    },
    "Okoye": {
        "Fortalezas": ["Habilidades de combate", "Lealtad"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Outrider": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Inteligencia limitada"]
    },
    "Pepper Potts (Extremis)": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Inestabilidad de Extremis"]
    },
    "Phastos": {
        "Fortalezas": ["Tecnología avanzada", "Inmortalidad"],
        "Debilidades": ["Desconocido"]
    },
    "Photon": {
        "Fortalezas": ["Manipulación de energía", "Vuelo"],
        "Debilidades": ["Desconocido"]
    },
    "Proxima Midnight": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia de su lanza"]
    },
    "Quake": {
        "Fortalezas": ["Manipulación de vibraciones"],
        "Debilidades": ["Vulnerabilidad física"]
    },
    "Quicksilver": {
        "Fortalezas": ["Velocidad sobrehumana"],
        "Debilidades": ["Impulsividad"]
    },
    "Red Guardian": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Humano con límites físicos"]
    },
    "Red Skull (Stonekeeper)": {
        "Fortalezas": ["Inmortalidad", "Conocimiento de las gemas del infinito"],
        "Debilidades": ["Encadenado a Vormir"]
    },
    "Red Skull": {
        "Fortalezas": ["Intelecto avanzado", "Habilidades de combate"],
        "Debilidades": ["Arrogancia"]
    },
    "Rescue": {
        "Fortalezas": ["Tecnología avanzada", "Vuelo"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Rocket Raccoon": {
        "Fortalezas": ["Inteligencia", "Habilidades de combate"],
        "Debilidades": ["Tamaño pequeño"]
    },
    "Ronan The Accuser (Power Stone)": {
        "Fortalezas": ["Fuerza sobrehumana", "Poder de la gema del poder"],
        "Debilidades": ["Dependencia de la gema del poder"]
    },
    "Ronan The Accuser": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Arrogancia"]
    },
    "Ronin": {
        "Fortalezas": ["Habilidades de combate", "Sigilo"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Scarlet Witch": {
        "Fortalezas": ["Magia del caos", "Manipulación de la realidad"],
        "Debilidades": ["Inestabilidad emocional"]
    },
    "Sersi (uni-mind)": {
        "Fortalezas": ["Manipulación de materia", "Inmortalidad"],
        "Debilidades": ["Dependencia del Uni-Mind"]
    },
    "Sersi": {
        "Fortalezas": ["Manipulación de materia", "Inmortalidad"],
        "Debilidades": ["Desconocido"]
    },
    "Shang-Chi": {
        "Fortalezas": ["Habilidades de combate", "Diez anillos"],
        "Debilidades": ["Desconocido"]
    },
    "She-Hulk": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Impulsividad"]
    },
    "Shuri": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Inexperiencia en combate"]
    },
    "Sif": {
        "Fortalezas": ["Habilidades de combate", "Longevidad"],
        "Debilidades": ["Desconocido"]
    },
    "Sinister Strange": {
        "Fortalezas": ["Magia avanzada", "Manipulación del tiempo"],
        "Debilidades": ["Corrupción por el Darkhold"]
    },
    "Smart Hulk": {
        "Fortalezas": ["Fuerza sobrehumana", "Inteligencia"],
        "Debilidades": ["Efectos secundarios del Gauntlet"]
    },
    "Spectral Vision": {
        "Fortalezas": ["Manipulación de densidad", "Vuelo"],
        "Debilidades": ["Crisis de identidad"]
    },
    "Spider-Man": {
        "Fortalezas": ["Fuerza sobrehumana", "Sentido arácnido"],
        "Debilidades": ["Inexperiencia"]
    },
    "Star-Lord (Celestial Power)": {
        "Fortalezas": ["Poder celestial", "Vuelo"],
        "Debilidades": ["Dependencia de su conexión celestial"]
    },
    "Star-Lord": {
        "Fortalezas": ["Habilidades de combate", "Tecnología avanzada"],
        "Debilidades": ["Desconocido"]
    },
    "Stature": {
        "Fortalezas": ["Cambio de tamaño", "Fuerza incrementada"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Surtur": {
        "Fortalezas": ["Fuerza sobrehumana", "Piroquinesis"],
        "Debilidades": ["Vulnerabilidad cuando su corona es removida"]
    },
    "Swole Groot": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Vulnerabilidad al fuego"]
    },
    "Talos The Untamed": {
        "Fortalezas": ["Cambio de forma", "Inteligencia"],
        "Debilidades": ["Desconocido"]
    },
    "Taskmaster": {
        "Fortalezas": ["Reflejos fotográficos", "Habilidades de combate"],
        "Debilidades": ["Desconocido"]
    },
    "Teenage Groot": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Vulnerabilidad al fuego"]
    },
    "Thanos": {
        "Fortalezas": ["Fuerza sobrehumana", "Inteligencia estratégica"],
        "Debilidades": ["Arrogancia"]
    },
    "Thanos (Infinity Gauntlet)": {
        "Fortalezas": ["Fuerza sobrehumana", "Poder de las gemas del infinito"],
        "Debilidades": ["Dependencia del guantelete"]
    },
    "The Collector": {
        "Fortalezas": ["Conocimiento extenso", "Recursos infinitos"],
        "Debilidades": ["Obsesión con coleccionar"]
    },
    "The Executioner": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Desconocido"]
    },
    "The Grandmaster": {
        "Fortalezas": ["Longevidad", "Manipulación de la materia"],
        "Debilidades": ["Arrogancia"]
    },
    "The Great Protector": {
        "Fortalezas": ["Fuerza sobrehumana", "Vuelo", "Control del agua"],
        "Debilidades": ["Desconocido"]
    },
    "The Punisher": {
        "Fortalezas": ["Habilidades de combate", "Resistencia"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Thena": {
        "Fortalezas": ["Fuerza sobrehumana", "Creación de armas de energía"],
        "Debilidades": ["Desconocido"]
    },
    "Thor (Pre-Ragnarok)": {
        "Fortalezas": ["Fuerza sobrehumana", "Control del rayo", "Vuelo"],
        "Debilidades": ["Arrogancia"]
    },
    "Thor (Awakened)": {
        "Fortalezas": ["Fuerza sobrehumana", "Control del rayo", "Vuelo"],
        "Debilidades": ["Dependencia emocional"]
    },
    "Thor": {
        "Fortalezas": ["Fuerza sobrehumana", "Control del rayo", "Vuelo"],
        "Debilidades": ["Dependencia emocional"]
    },
    "Ultron": {
        "Fortalezas": ["Inteligencia avanzada", "Fuerza sobrehumana"],
        "Debilidades": ["Arrogancia"]
    },
    "Unworthy Thor": {
        "Fortalezas": ["Fuerza sobrehumana", "Control del rayo", "Vuelo"],
        "Debilidades": ["Baja autoestima"]
    },
    "Valkyrie": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Trauma emocional"]
    },
    "Vision": {
        "Fortalezas": ["Manipulación de densidad", "Vuelo", "Energía del rayo"],
        "Debilidades": ["Dependencia de la gema de la mente"]
    },
    "Volstagg": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Desconocido"]
    },
    "Vulture": {
        "Fortalezas": ["Tecnología avanzada", "Vuelo"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Wanda Maximoff (Pre-WandaVision)": {
        "Fortalezas": ["Magia del caos", "Manipulación de la realidad"],
        "Debilidades": ["Inestabilidad emocional"]
    },
    "War Machine": {
        "Fortalezas": ["Tecnología avanzada", "Vuelo"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Warlock": {
        "Fortalezas": ["Magia avanzada", "Inmortalidad"],
        "Debilidades": ["Desconocido"]
    },
    "Wasp": {
        "Fortalezas": ["Cambio de tamaño", "Vuelo"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Whiplash": {
        "Fortalezas": ["Tecnología avanzada", "Fuerza incrementada"],
        "Debilidades": ["Dependencia de la tecnología"]
    },
    "Whiplash (Mark II)": {
        "Fortalezas": ["Tecnología avanzada", "Fuerza incrementada"],
        "Debilidades": ["Dependencia de la tecnología"]
    },
    "White Wolf": {
        "Fortalezas": ["Habilidades de combate", "Estrategia"],
        "Debilidades": ["Trauma psicológico"]
    },
    "Winter Soldier": {
        "Fortalezas": ["Fuerza sobrehumana", "Brazo biónico", "Habilidades de combate"],
        "Debilidades": ["Programación mental"]
    },
    "Wolverine": {
        "Fortalezas": ["Regeneración", "Fuerza sobrehumana"],
        "Debilidades": ["Vulnerabilidad emocional"]
    },
    "Wong": {
        "Fortalezas": ["Magia avanzada", "Conocimiento místico"],
        "Debilidades": ["Desconocido"]
    },
    "YellowJacket": {
        "Fortalezas": ["Cambio de tamaño", "Fuerza incrementada"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Yondu": {
        "Fortalezas": ["Control sobre su flecha", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Zeus": {
        "Fortalezas": ["Poder divino", "Inmortalidad"],
        "Debilidades": ["Arrogancia"]
    }
}

In [68]:
# Asignar fortalezas y debilidades
def assign_strenghs_weaknesses_MCU(row):
    hero_name = row['nombre']
    if hero_name in strengths_weaknesses_MCU:
        return pd.Series([
            strengths_weaknesses_MCU[hero_name]['Fortalezas'],
            strengths_weaknesses_MCU[hero_name]['Debilidades']
        ])
    else:
        return pd.Series([['Desconocido'], ['Desconocido']])

In [69]:
mcu_df[['Fortalezas', 'Debilidades']] = mcu_df.apply(assign_strenghs_weaknesses_MCU, axis = 1)

In [73]:
mcu_df.info()
mcu_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre_completo  161 non-null    object
 1   nombre           176 non-null    object
 2   universo         176 non-null    object
 3   url              176 non-null    object
 4   Intelligence     176 non-null    int64 
 5   Strength         176 non-null    int64 
 6   Speed            176 non-null    object
 7   Durability       176 non-null    int64 
 8   Power            176 non-null    int64 
 9   Combat           176 non-null    int64 
 10  Tier             176 non-null    int64 
 11  Tier2            48 non-null     object
 12  superpoderes     176 non-null    object
 13  heroe            176 non-null    object
 14  Fortalezas       176 non-null    object
 15  Debilidades      176 non-null    object
dtypes: int64(6), object(10)
memory usage: 22.1+ KB


,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,superpoderes,heroe,Fortalezas,Debilidades
0,NaN,Agatha Harkness,Marvel Cinematic Universe,https://www.superherodb.com/agatha-harkness/10...,70,40,25,35,70,65,3,NaN,"['Durability', 'Energy Blasts', 'Energy Constr...",Agatha Harkness,"[Magia oscura, Manipulación mágica]",[Necesita absorber la magia de otros para mant...
1,Emil Blonsky,Abomination,Marvel Cinematic Universe,https://www.superherodb.com/abomination/10-12512/,65,100,25,50,65,70,3,3,"['Cold Resistance', 'Durability', 'Endurance',...",Abomination,"[Fuerza sobrehumana, Resistencia]",[Vulnerable a ataques mentales]
2,Ajak,Ajak,Marvel Cinematic Universe,https://www.superherodb.com/ajak/10-26887/,65,25,19,25,50,70,2,NaN,"['Accelerated Healing', 'Agility', 'Durability...",Ajak,"[Curación, Fuerza y durabilidad sobrehumanas]",[Vulnerable a traiciones de otros Eternos]
3,NaN,Ancient One (time stone),Marvel Cinematic Universe,https://www.superherodb.com/ancient-one-time-s...,85,8,25,15,100,100,6,6++,"['Anti-Gravity', 'Astral Projection', 'Concept...",Ancient One (Time Stone),"[Manipulación del tiempo, Magia avanzada]",[Dependencia de la Piedra del Tiempo]
4,NaN,Ancient One,Marvel Cinematic Universe,https://www.superherodb.com/ancient-one/10-12735/,85,8,25,25,90,100,4,NaN,"['Anti-Gravity', 'Astral Projection', 'Dimensi...",Ancient One,"[Magia avanzada, Longevidad]",[Vulnerabilidad física sin la Piedra del Tiempo]


In [26]:
# Extraer nombres de los héroes
heroes = df_DC['heroe'].tolist()

In [27]:
# Generar combinaciones ficticias de enfrentamientos
enfrentamientos = list(itertools.combinations(heroes, 2))

In [28]:
# Crear el dataframe de enfrentamientos
enfrentamientos_df = pd.DataFrame(enfrentamientos, columns=['Heroe_1', 'Heroe_2'])


BATALLAS Y RESULTADO

In [30]:
battle_DC = pd.read_csv('./data/enfrentamientos_heroes_dc.csv')

In [31]:
battle_DC

,Heroe_1,Heroe_2
0,Aquaman,Ares
1,Aquaman,Atom Smasher
2,Aquaman,Batman
3,Aquaman,Black Adam
4,Aquaman,Black Canary
...,...,...
2623,White Dragon,Wrath
2624,White Dragon,Zeus
2625,Wonder Woman,Wrath
2626,Wonder Woman,Zeus


In [34]:
batallas_resultados = pd.read_csv('./data/enfrentamientos_heroes_dc_con_resultados_narrativos.csv')

In [35]:
batallas_resultados.head()

,Heroe_1,Heroe_2,Resultado_x,Resultado_y,Resultado
0,Aquaman,Ares,2,2.0,2.0
1,Aquaman,Atom Smasher,1,1.0,1.0
2,Aquaman,Batman,1,1.0,2.0
3,Aquaman,Black Adam,2,2.0,2.0
4,Aquaman,Black Canary,1,1.0,1.0


In [36]:
batallas_resultados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2628 entries, 0 to 2627
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Heroe_1      2628 non-null   object 
 1   Heroe_2      2628 non-null   object 
 2   Resultado_x  2628 non-null   int64  
 3   Resultado_y  5 non-null      float64
 4   Resultado    5 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 102.8+ KB


BATALLAS DC COMPLETO

In [37]:
# Data battle_df
battle_df = pd.read_csv('./data/enfrentamientos_heroes_dc_con_resultados_narrativos.csv')
battle_df.head()

,Heroe_1,Heroe_2,Resultado_x,Resultado_y,Resultado
0,Aquaman,Ares,2,2.0,2.0
1,Aquaman,Atom Smasher,1,1.0,1.0
2,Aquaman,Batman,1,1.0,2.0
3,Aquaman,Black Adam,2,2.0,2.0
4,Aquaman,Black Canary,1,1.0,1.0


In [38]:
# Data hero_atribute_df
hero_atribute_df = pd.read_csv('./data/heroes_dc_extended')
hero_atribute_df.head()

,Unnamed: 0,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,heroe
0,0,Arthur Curry,Aquaman,DC Extended Universe,https://www.superherodb.com/aquaman/10-12533/,65,50,25,30,45,80,3,NaN,Aquaman
1,1,Ares,Ares,DC Extended Universe,https://www.superherodb.com/ares/10-12597/,90,100,50,50,70,100,4,4,Ares
2,2,Albert Rothstein,Atom Smasher,DC Extended Universe,https://www.superherodb.com/atom-smasher/10-27...,60,30,25,25,40,20,2,NaN,Atom Smasher
3,3,Bruce Wayne,Batman,DC Extended Universe,https://www.superherodb.com/batman/10-12530/,80,13,11,15,20,95,1,NaN,Batman
4,4,Teth-Adam,Black Adam,DC Extended Universe,https://www.superherodb.com/black-adam/10-21003/,65,100,75,45,75,75,3,3,Black Adam


In [46]:
# Data final battle_df + hero_atribute_df

# Merge de los dos datas 
battle_merged_df = battle_df.merge(hero_atribute_df, left_on = 'Heroe_1', right_on='heroe', how='left')
battle_merged_df = battle_merged_df.merge(hero_atribute_df, left_on='Heroe_2', right_on='heroe', how='left', suffixes=('_1', '_2'))

# Data final_battle_df
final_battle_df = battle_merged_df[['Heroe_1', 'Heroe_2', 'universo_1', 'Intelligence_1', 'Strength_1', 'Speed_1', 
                                             'Durability_1', 'Power_1', 'Combat_1',
                                             'universo_2', 'Intelligence_2', 'Strength_2', 'Speed_2', 
                                             'Durability_2', 'Power_2', 'Combat_2', 'Resultado_x']]


In [47]:
final_battle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2628 entries, 0 to 2627
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Heroe_1         2628 non-null   object
 1   Heroe_2         2628 non-null   object
 2   universo_1      2628 non-null   object
 3   Intelligence_1  2628 non-null   int64 
 4   Strength_1      2628 non-null   int64 
 5   Speed_1         2628 non-null   int64 
 6   Durability_1    2628 non-null   int64 
 7   Power_1         2628 non-null   int64 
 8   Combat_1        2628 non-null   int64 
 9   universo_2      2628 non-null   object
 10  Intelligence_2  2628 non-null   int64 
 11  Strength_2      2628 non-null   int64 
 12  Speed_2         2628 non-null   int64 
 13  Durability_2    2628 non-null   int64 
 14  Power_2         2628 non-null   int64 
 15  Combat_2        2628 non-null   int64 
 16  Resultado_x     2628 non-null   int64 
dtypes: int64(13), object(4)
memory usage: 349.2+ KB


In [49]:
final_battle_df.head()

,Heroe_1,Heroe_2,universo_1,Intelligence_1,Strength_1,Speed_1,Durability_1,Power_1,Combat_1,universo_2,Intelligence_2,Strength_2,Speed_2,Durability_2,Power_2,Combat_2,Resultado_x
0,Aquaman,Ares,DC Extended Universe,65,50,25,30,45,80,DC Extended Universe,90,100,50,50,70,100,2
1,Aquaman,Atom Smasher,DC Extended Universe,65,50,25,30,45,80,DC Extended Universe,60,30,25,25,40,20,1
2,Aquaman,Batman,DC Extended Universe,65,50,25,30,45,80,DC Extended Universe,80,13,11,15,20,95,1
3,Aquaman,Black Adam,DC Extended Universe,65,50,25,30,45,80,DC Extended Universe,65,100,75,45,75,75,2
4,Aquaman,Black Canary,DC Extended Universe,65,50,25,30,45,80,DC Extended Universe,55,1,6,5,35,40,1


In [50]:
final_battle_df[final_battle_df['Heroe_1'] == 'The Flash']

,Heroe_1,Heroe_2,universo_1,Intelligence_1,Strength_1,Speed_1,Durability_1,Power_1,Combat_1,universo_2,Intelligence_2,Strength_2,Speed_2,Durability_2,Power_2,Combat_2,Resultado_x
2613,The Flash,Vigilante,DC Extended Universe,80,25,100,30,85,40,DC Extended Universe,60,1,7,10,10,70,1
2614,The Flash,White Dragon,DC Extended Universe,80,25,100,30,85,40,DC Extended Universe,65,13,25,10,20,40,1
2615,The Flash,Wonder Woman,DC Extended Universe,80,25,100,30,85,40,DC Extended Universe,80,95,70,60,55,100,2
2616,The Flash,Wrath,DC Extended Universe,80,25,100,30,85,40,DC Extended Universe,65,65,65,40,60,65,0
2617,The Flash,Zeus,DC Extended Universe,80,25,100,30,85,40,DC Extended Universe,95,100,25,50,80,85,2


In [51]:
final_battle_df.iloc[150:160]

,Heroe_1,Heroe_2,universo_1,Intelligence_1,Strength_1,Speed_1,Durability_1,Power_1,Combat_1,universo_2,Intelligence_2,Strength_2,Speed_2,Durability_2,Power_2,Combat_2,Resultado_x
150,Atom Smasher,Cassandra Cain,DC Extended Universe,60,30,25,25,40,20,DC Extended Universe,45,1,1,1,1,30,1
151,Atom Smasher,Cheetah,DC Extended Universe,60,30,25,25,40,20,DC Extended Universe,60,50,14,15,30,65,2
152,Atom Smasher,Cyborg,DC Extended Universe,60,30,25,25,40,20,DC Extended Universe,70,40,30,35,65,50,2
153,Atom Smasher,Cyclone,DC Extended Universe,60,30,25,25,40,20,DC Extended Universe,80,1,25,15,50,40,2
154,Atom Smasher,Dark Flash,DC Extended Universe,60,30,25,25,40,20,DC Extended Universe,80,25,100,35,85,50,2
155,Atom Smasher,Darkseid,DC Extended Universe,60,30,25,25,40,20,DC Extended Universe,95,100,30,55,90,95,2
156,Atom Smasher,Deadshot,DC Extended Universe,60,30,25,25,40,20,DC Extended Universe,60,1,6,7,17,45,1
157,Atom Smasher,Doctor Fate,DC Extended Universe,60,30,25,25,40,20,DC Extended Universe,85,75,60,45,85,70,2
158,Atom Smasher,Doctor Sivana,DC Extended Universe,60,30,25,25,40,20,DC Extended Universe,70,95,60,40,70,55,2
159,Atom Smasher,Doomsday,DC Extended Universe,60,30,25,25,40,20,DC Extended Universe,35,100,45,45,75,25,2
